# Measurement of an Impulse Response with a Microphone

This notebook scetches how to measure impulse responses with a single mircophone. 

* Two sweeps are played, one for the loudspeaker and one for the analog feedback. 
* Two channels are recorded, the microphone and the analog feedback.

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2

# Load requirements
import sys

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from IPython import display
from numpy.fft import rfft as fft, rfftfreq as fftfreq
from pathlib import Path
from time import sleep

import acoustics_hardware.serial

# Set up hardware

In [ ]:
print(acoustics_hardware.devices.AudioDevice.get_devices())

device_name = 'Orion 32'
fs = 48000

try:
    device = acoustics_hardware.devices.AudioDevice(
        name=device_name,
        fs=fs,
    )
except ValueError as e:
    sys.exit(e)

# 2 inputs (0..31) for 'Orion 32'
device.add_input(0)
device.add_input(1)

# 2 outputs (0..31) for 'Orion 32'
device.add_output(0)
device.add_output(1)

amplitude_db = -20
sweep_duration = 3 # sec

amplitude_lin = 10**(amplitude_db/20)

# Signal is automatically played through the first 2 channels
generator = acoustics_hardware.generators.SweepGenerator(device=device,
    start_frequency=50, stop_frequency=20e3, duration=sweep_duration, repetitions=1, amplitude=amplitude_lin)

# Define functions for plotting

In [ ]:
# noinspection PyShadowingNames
def plot_data_t(data):
    for ch in np.arange(data.shape[0]): # individual channels
        plt.plot(data[ch, :])
        plt.xlim(0, data.shape[-1])
        plt.xlabel('t (taps)')
        plt.legend([f'Channel {ch}'], loc='upper right')
        plt.grid()
        plt.show()
        
# noinspection PyShadowingNames
def plot_data_f(data, fs):
    freq_data = fft(data)
    f = fftfreq(data.shape[1], 1/fs)
    
    for ch in np.arange(data.shape[0]): # individual channels
        plt.semilogx(f, 20*np.log10(abs(freq_data[ch-1, :])).T)
        plt.xlim(30, fs/2)
        plt.xlabel('f (Hz)')
        plt.legend([f'Channel {ch}'], loc='upper right')
        plt.grid()
        plt.show() 

# Perform measurement

In [ ]:
out_file_name = 'recorded_data.mat'

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

# Wait before starting the measurement
sleep(1)

device.reset()
device.start(timed=sweep_duration + 1, blocking=True)
data = device.get_input_data()

# Store the data
sio.savemat(
    out_file_name,
    {
        'recorded_signal': np.transpose(data),
        'fs': fs,
    },
)

# Play audio
display.display(
    display.Markdown('### Channel 1'),
    display.Audio(data[0,:], rate=fs),
    display.Markdown('### Channel 2'),
    display.Audio(data[1,:], rate=fs))

plot_data_t(data)
plot_data_f(data, fs) 